In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2070 SUPER'

In [4]:
torch.cuda.mem_get_info()

(7755792384, 8361738240)

In [5]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

if device.type=='cuda':
    print(torch.cuda.get_device_name(0))

cuda
NVIDIA GeForce RTX 2070 SUPER


In [6]:
CLASSES=10
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [8]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=96, kernel_size=11, stride=4, padding=0)

        self.conv2 = nn.Conv2d(in_channels=96, out_channels=96, kernel_size=1)

        self.conv3 = nn.Conv2d(in_channels=96,out_channels=96, kernel_size=1)
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.drop3 = nn.Dropout(0.5)

        self.conv4 = nn.Conv2d(in_channels=96,out_channels=256, kernel_size=11, stride=4, padding=2)

        self.conv5 = nn.Conv2d(in_channels=256,out_channels=256, kernel_size=1)
        self.pool5 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.drop5 = nn.Dropout(0.5)

        self.conv6 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)

        self.conv7 = nn.Conv2d(384, 384, kernel_size=1)

        self.conv8 = nn.Conv2d(384, 384, kernel_size=1)
        self.drop8 = nn.Dropout(0.5)

        self.conv9 = nn.Conv2d(384, 10, kernel_size=3, stride=1, padding=1)

        self.conv10 = nn.Conv2d(10, 10, kernel_size=1)

        self.conv11 = nn.Conv2d(10, 10, kernel_size=1)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.drop3(self.pool3(F.relu(self.conv3(x))))
        x = F.relu(self.conv4(x))
        x = self.drop5(self.pool5(F.relu(self.conv5(x))))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = self.drop8(F.relu(self.conv8(x)))
        x = F.relu(self.conv9(x))
        x = F.relu(self.conv10(x))
        x = F.relu(self.conv11(x))
        x = self.adaptive_pool(x)
        x = torch.flatten(x, 1)
        return x

In [9]:
transform_conf=transforms.Compose([
    transforms.Resize((227,227)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [10]:
BATCH_SIZE = 16
train_dataset = datasets.MNIST(
    "/workspaces/CS490Assignment2/data",
    train=True,
    download=True,
    transform=transform_conf,
)
test_dataset = datasets.MNIST(
    "/workspaces/CS490Assignment2/data",
    train=False,
    download=True,
    transform=transform_conf,
)

In [11]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True
)

In [12]:
model = Net().to(device)
optimizer = optim.Adam(params=model.parameters(), lr=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [13]:
def train(model, device, train_loader, optimizer, epochs):
    print("inside train")
    model.train()
    for batch_ids, (img, classes) in enumerate(train_loader):
        classes = classes.type(torch.LongTensor)
        img, classes = img.to(device), classes.to(device)
        torch.autograd.set_detect_anomaly(True)
        optimizer.zero_grad()
        output = model(img)
        loss = loss_fn(output, classes)

        loss.backward()
        optimizer.step()
    if (batch_ids + 1) % 2 == 0:
        print(
            "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epochs,
                batch_ids * len(img),
                len(train_loader.dataset),
                100.0 * batch_ids / len(train_loader),
                loss.item(),
            )
        )

In [14]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for img, classes in test_loader:
            img, classes = img.to(device), classes.to(device)
            y_hat = model(img)
            test_loss += F.nll_loss(y_hat, classes, reduction="sum").item()
            _, y_pred = torch.max(y_hat, 1)
            correct += (y_pred == classes).sum().item()
        test_loss /= len(test_dataset)
        print(
            "\n Test set: Avarage loss: {:.0f},Accuracy:{}/{} ({:.0f}%)\n".format(
                test_loss,
                correct,
                len(test_dataset),
                100.0 * correct / len(test_dataset),
            )
        )
        print("=" * 30)

In [15]:
if __name__ == "__main__":
    seed = 42
    EPOCHS = 2

    for epoch in range(1, EPOCHS + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

inside train
Train Epoch: 1 [59984/60000 (100%)]	Loss: 0.486223

 Test set: Avarage loss: -14,Accuracy:9629/10000 (96%)

inside train
Train Epoch: 2 [59984/60000 (100%)]	Loss: 0.023493

 Test set: Avarage loss: -16,Accuracy:9763/10000 (98%)

